In [8]:
logFiles = "/loudacre/weblogs/*2.log"
userCountRDD = sc.textFile(logFiles)\
    .map(lambda line: (line.split(' ')[2], 1))\
    .reduceByKey(lambda v1,v2: v1+v2)

In [9]:
userCountRDD.take(2)

[(u'3922', 6), (u'104959', 2)]

In [10]:
countRDD = userCountRDD.map(lambda (userId, count): (count, userId))

In [11]:
countRDD.take(2)

[(6, u'3922'), (2, u'104959')]

In [12]:
countMap = countRDD.countByKey()

In [13]:
print countMap

defaultdict(<type 'int'>, {128: 9, 2: 7239, 3: 36, 4: 4155, 5: 26, 6: 2162, 7: 14, 8: 1409, 9: 14, 10: 878, 11: 12, 12: 549, 13: 7, 14: 308, 15: 8, 16: 155, 17: 4, 146: 8, 19: 2, 20: 41, 21: 1, 22: 17, 150: 11, 152: 11, 132: 12, 154: 5, 27: 1, 156: 5, 158: 6, 160: 8, 162: 5, 164: 3, 134: 9, 166: 3, 168: 4, 170: 3, 172: 6, 174: 2, 24: 6, 176: 2, 136: 11, 178: 1, 188: 1, 138: 6, 190: 1, 130: 10, 140: 14, 142: 8, 86: 1, 144: 7, 100: 1, 104: 1, 106: 1, 18: 76, 110: 4, 112: 1, 116: 1, 118: 4, 120: 5, 148: 2, 122: 4, 124: 7, 126: 5})


In [16]:
userIdIpRDD = sc.textFile(logFiles)\
    .map(lambda line: line.split(' '))\
    .map(lambda fields: (fields[2], fields[0]))\
    .groupByKey()

In [17]:
userIdIpRDD.take(2)

[(u'3922', <pyspark.resultiterable.ResultIterable at 0x7f39489705d0>),
 (u'104959', <pyspark.resultiterable.ResultIterable at 0x7f3948989590>)]

In [20]:
accountFiles = "/loudacre/accounts/*"
accountRDD = sc.textFile(accountFiles)\
    .map(lambda line: line.split(','))\
    .map(lambda fields: (fields[0], fields))

In [21]:
accountRDD.take(2)

[(u'1',
  [u'1',
   u'2008-10-23 16:05:05.0',
   u'\\N',
   u'Donald',
   u'Becton',
   u'2275 Washburn Street',
   u'Oakland',
   u'CA',
   u'94660',
   u'5100032418',
   u'2014-03-18 13:29:47.0',
   u'2014-03-18 13:29:47.0']),
 (u'2',
  [u'2',
   u'2008-11-12 03:00:01.0',
   u'\\N',
   u'Donna',
   u'Jones',
   u'3885 Elliott Street',
   u'San Francisco',
   u'CA',
   u'94171',
   u'4150835799',
   u'2014-03-18 13:29:47.0',
   u'2014-03-18 13:29:47.0'])]

In [22]:
accountHitRDD = accountRDD.join(userCountRDD)

In [23]:
accountHitRDD.take(2)

[(u'89371',
  ([u'89371',
    u'2013-09-08 02:21:15.0',
    u'2014-01-19 12:17:06.0',
    u'Ricky',
    u'Pope',
    u'4535 Highland Drive',
    u'Portland',
    u'OR',
    u'97212',
    u'5033136196',
    u'2014-03-18 13:32:36.0',
    u'2014-03-18 13:32:36.0'],
   4)),
 (u'99996',
  ([u'99996',
    u'2013-03-14 19:19:45.0',
    u'2014-02-07 16:32:29.0',
    u'Garrett',
    u'Allen',
    u'495 Wilson Street',
    u'Prescott',
    u'AZ',
    u'86360',
    u'9280545713',
    u'2014-03-18 13:32:56.0',
    u'2014-03-18 13:32:56.0'],
   2))]

In [28]:
for (userid, (accounts, hit)) in accountHitRDD.take(5):
    print userid,hit,accounts[3],accounts[4]

89371 4 Ricky Pope
99996 2 Garrett Allen
69171 6 Richard Tarver
90311 2 David Rosenberg
36848 6 Aaron Hutson


In [38]:
postalRDD = sc.textFile(accountFiles)\
    .keyBy(lambda line: line.split(',')[8])\
    .mapValues(lambda value: value.split(',')[3] + "," + value.split(',')[4])\
    .groupByKey()\
    .sortByKey()

In [39]:
for (postalCode, userNames) in postalRDD.take(5):
    print '---',postalCode
    for (userName) in userNames:
        print userName

--- 85000
Harvey,Allen
Daniel,Prinz
Robert,Pascale
Donna,Brookes
James,Mackenzie
Robert,Chamberlain
Richard,Cunningham
Bailey,Sewell
Daniel,Marin
--- 85001
Frances,Mendelsohn
Mary,Watson
Donald,Brookover
Brandon,Hathaway
Crystal,Leonard
Carrie,Moran
Marie,Kirksey
Issac,Lance
Vesta,Barnes
Eva,Fiore
Keith,Tucker
Danielle,Medford
Norman,Spell
Shelley,Soto
Kathy,Frantz
Timothy,Wilkins
Joseph,Snyder
Delbert,Flores
Gail,Eakes
Bert,Daniels
Vincent,Carpenter
--- 85002
Ruby,Whitney
David,Perry
Marianne,James
Nancy,Holiman
Allen,Roman
Donna,Manus
Nancy,Reed
Estella,Baird
James,Gilbert
David,McKay
Laura,Clark
John,Horn
Jessica,Payne
Bryant,Stewart
Jose,Jones
Wesley,Robinson
--- 85003
Mark,Martin
Vivian,Ross
Harry,Tabor
Kyle,Strickland
Kevin,Dvorak
Virginia,Wisniewski
Catherine,Gibson
Lindsey,Thies
--- 85004
Mary,Kitts
Kevin,Viola
Tonya,Meadows
Sherry,Royalty
Greg,Collins
Joseph,Shirley
Sandra,White
Timothy,Stern
Dominic,Johnson
Mary,Dewitt
Matthew,Carpenter
Annie,Ball
Kathleen,Pate
Carrie,Lish
